## UTAH FORGE WELL 58-32

**Well 58-32 was drilled to a depth of 7536 feet** in the Milford FORGE area during the summer of
2017 to confirm the reservoir characteristics inferred from existing wells and a wide variety of
both new and legacy geologic and geophysical data. **Drill cuttings were collected and described
at 10-foot intervals** and a robust **suite of geophysical logs** were run. Analyses show
that the basement rock within the FORGE area consists of a suite of **intrusive rock types that are
primarily granitic. Some diorite and monzodiorite was also encountered**, as was a significant
volume of rock with a more intermediate composition.

The density of the granite and intermediate rock types typically range from **2.6 to
2.65 g/cm³**, but the higher gamma response of the **granitic rock (140–290 gAPI)** can often
differentiate granitic compositions from **intermediate compositions (70–210 gAPI).** The **higher
density (2.7–3.0 g/cm³) and lower gamma values (50–80 gAPI) of the dioritic compositions** is
more distinctive and greatly simplifies identification. 

The various laboratory analyses and geophysical logs of the 58-32 well prove it was drilled into **low porosity/low permeability intrusive rock** with temperatures well within the U.S. Department of Energy-specified window of **175°–225°C (347°–437°F).** More details here https://utahforge.com/
 

### Let's import the libraries, remember Lasio and Seaborn must be installed previously

In [ ]:
import lasio
import pandas as pd
import numpy as np
#libraries for plots
import matplotlib.pyplot as plt
import seaborn as sns
#to avoid warnings to show up
import warnings
warnings.filterwarnings('ignore')

### Read 58-32 well logs with Lasio and inspect

In [ ]:
reg_all = lasio.read('../alldata/58-32_main.las')

In [ ]:
reg_all.version

In [ ]:
reg_all.curves

In [ ]:
reg_all.data

In [ ]:
reg_all.keys()

In [ ]:
reg_all['SP']

### From Lasio to Data Frame Pandas
DataFrames in Pandas are two-dimensional tables with row and columns that can be easily edited and manipulated. 

In [ ]:
df_main = reg_all.df()
df_main

In [ ]:
#Print the first 5 rows of the data frame with the header of the columns
df_main.head(5)

In [ ]:
#Print the last 10 rows of the data frame with the header of the columns
df_main.tail(10)

In [ ]:
# statistics
df_main.describe()

In [ ]:
#parameters from only 1 column 
df_main.AF10.std()

### Create a dataset only with GR, SP, AT10, AT90, RHOZ, NPHI, CTEM

In [ ]:
df_mini = df_main[['GR', 'SP', 'AT10', 'AT90', 'RHOZ', 'NPHI', 'CTEM']]

In [ ]:
df_mini.describe()

In [ ]:
df_mini['CTEM_C']= ((df_mini['CTEM']-32)*5)/9

In [ ]:
df_mini.info()

In [ ]:
count_neg = (df_mini.RHOZ < 0).sum()
count_neg

In [ ]:
df_mini.loc[(df_mini['RHOZ'] < 0), 'RHOZ']=np.nan

In [ ]:
count_neg = (df_mini.RHOZ < 0).sum()
count_neg

**Unknown LowGR (<50)
**Dioritic comp. (50–80 gAPI)
**Intermediate comp. (80–140 gAPI)
**Granite (140–290 gAPI)
**Unknown HighGR(>290)

In [ ]:
conditions = [
    (df_mini['GR'] <= 50),
    (df_mini['GR'] > 50) & (df_mini['GR'] <= 80),
    (df_mini['GR'] > 80) & (df_mini['GR'] <= 140),
    (df_mini['GR'] > 140) & (df_mini['GR'] <= 290),
    (df_mini['GR'] > 290)
    ]

# create a list of the values we want to assign for each condition
values = ['Unknown LowGR', 'Dioritic Comp', 'Intermediate Comp', 'Granite', 'Unknown HighGR' ]

# create a new column and use np.select to assign values to it using our lists as arguments
df_mini['Labels'] = np.select(conditions, values)

In [ ]:
df_mini.sample(10)

In [ ]:
#statistics grouped by Labels 

df_mini[['Labels','GR', 'SP', 'AT10', 'AT90', 'RHOZ', 'NPHI', 'CTEM', 'CTEM_C']].groupby('Labels').mean()

### Read Thermal conductivity and mineralogy data measured in drill cuttings. CAUTION: Depths are in meters, need to be converted to feet
##### Full report https://ugspub.nr.utah.gov/publications/misc_pubs/mp-169/mp-169-l.pdf

In [ ]:
TC_coredata = pd.read_csv ('../alldata/58-32_thermal_conductivity_data.csv', index_col=1)
XRD_coredata = pd.read_csv ('../alldata/58-32_xray_diffraction_data.csv', index_col=1)

#TC_coredata.head()
XRD_coredata.columns

In [ ]:
TC_coredata.index

In [ ]:
XRD_coredata.index

In [ ]:
result = pd.concat([XRD_coredata, TC_coredata], axis=1, sort=False)

In [ ]:
result.columns

In [ ]:
cutt_data = result[['Illite','Plagioclase', 'K-feldspar', 
                    'Quartz', 'matrix thermal conductivity (W/m deg C)']]

In [ ]:
cutt_data.index=(3.28084*cutt_data.index) #m to ft
#cutt_data.loc[(cutt_data =='tr')]=np.nan
cutt_data=cutt_data.replace('tr', np.nan)
cutt_data.columns=['Illi', 'Plag', 'K-feld', 'Qz', 'TC']


In [ ]:
cutt_data.info()

In [ ]:
cutt_data.sample(5)

# Visualization 

In [ ]:
#let's start with something simple

In [ ]:
plt.hist(df_mini['_'],bins=__, range=__, histtype='_', orientation='_', color='_');

In [ ]:
plt.hist(df_mini[__],bins=__, range=__, histtype='__, orientation=___, color=__);

In [ ]:
sns.scatterplot(x=df_mini['__'], y=df_mini['___']);

In [ ]:
sns.scatterplot(x=df_mini['RHOZ'], y=df_mini['GR'], hue=df_mini['__']);

In [ ]:
sns.relplot(data=___,
    x="RHOZ", y="GR", col="Labels",
    hue="Labels");

In [ ]:
sns.countplot(y=df_mini['Labels']);

In [ ]:
sns.set(style="ticks", context="talk")
plt.style.use('___')
g = sns.PairGrid(df_mini.iloc[:,1:], diag_sharey=False, corner=True);
g.map_lower(sns.scatterplot);
g.map_diag(sns.kdeplot,shade=True, shade_lowest=False);

In [ ]:
#creating correlation matrix data
corr = df_mini.corr() #exclude any string data type
#figure parameters
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(___, ax=ax, cmap="___")
#plt.grid()
plt.show()

In [ ]:
#Creating 2 tracks to plot data form this particular dataframe
fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, squeeze=True, figsize=(15, 15), gridspec_kw={'wspace': 0.25})
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.975, top=0.7, wspace=0.5, hspace=0.2)
axs[0].set_ylabel('Depth (ft)')
axs[0].invert_yaxis()
axs[0].get_xaxis().set_visible(False)


In [ ]:
#Creating 2 tracks to plot data form this particular dataframe
fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, squeeze=True, figsize=(15, 15), gridspec_kw={'wspace': 0.25})
fig.subplots_adjust(left=0.05, bottom=0.05, right=0.975, top=0.7, wspace=1, hspace=0.2)
axs[0].set_ylabel('Depth (ft)')
axs[0].invert_yaxis()
axs[0].get_xaxis().set_visible(False)

ax1 = axs[0].twiny()  
ax1.plot(df_mini.GR, df_mini.index, '-', color='#2ea869', linewidth=0.5)
ax1.set_xlim(0,450) 
ax1.set_xlabel('GR (API)', color='#2ea869')
ax1.minorticks_on()
ax1.spines['top'].set_position(('axes', 1.0)) 

ax2 = axs[1].twiny() 
ax2.plot(df_mini.SP, df_mini.index, '-', color='#0a0a0a', linewidth=0.7)
ax2.set_xlim(-200,200) 
ax2.set_xlabel('SP(mV)', color='#0a0a0a')
ax2.minorticks_on()
ax2.spines['top'].set_position(('axes', 1.0))  

#Creating the Grid
ax2.grid(True)
axs[1].get_xaxis().set_visible(False)

### Create a function that would create a layout with basic logs and core data

In [ ]:
#Log visualization function, to be able to apply it to any set of data that has been converted to a dataFrame

def make_layout_tc (log_df, cuttings_df):

  
    fig, axs = plt.subplots(nrows=1, ncols=5, sharey=True, squeeze=True, figsize=(15, 15), gridspec_kw={'wspace': 0.25})
    fig.subplots_adjust(left=0.05, bottom=0.05, right=0.975, top=0.7, wspace=0.2, hspace=0.2)
    axs[0].set_ylabel('Depth (ft)')
    axs[0].invert_yaxis()
    axs[0].get_xaxis().set_visible(False)
    
    # First track GR/SP logs to display
    ax1 = axs[0].twiny()  
    ax1.plot(log_df.GR, log_df.index, '-', color='#2ea869', linewidth=0.5)
    ax1.set_xlim(0,450) 
    ax1.set_xlabel('GR (API)', color='#2ea869')
    ax1.minorticks_on()
    ax1.spines['top'].set_position(('axes', 1.075)) 
    
    ax2 = axs[0].twiny() 
    ax2.plot(log_df.SP, log_df.index, '-', color='#0a0a0a', linewidth=0.7)
    ax2.set_xlim(-200,200) 
    ax2.set_xlabel('SP(mV)', color='#0a0a0a')
    ax2.minorticks_on()
    ax2.spines['top'].set_position(('axes', 1.0))    
    ax2.grid(True)
    axs[0].get_xaxis().set_visible(False)
    
    # Second track RHOZ/NPHI logs to display
    ax1 = axs[1].twiny()  
    ax1.plot(log_df.RHOZ, log_df.index, '-', color='#ea0606', linewidth=0.5)
    ax1.set_xlim(1.5,3.0) 
    ax1.set_xlabel('RHOZ (g/cm3)', color='#ea0606')
    ax1.minorticks_on()
    ax1.spines['top'].set_position(('axes', 1.075))

    ax2 = axs[1].twiny()  
    ax2.plot(log_df.NPHI, log_df.index, '-', color='#1577e0', linewidth=0.5)
    ax2.set_xlim(1,0) 
    ax2.set_xlabel('NPHI (v/v)', color='#1577e0')
    ax2.minorticks_on()
    ax2.spines['top'].set_position(('axes', 1.0))
    ax2.grid(True)
    axs[1].get_xaxis().set_visible(False)
    
    # Third track Resistivities
    ax1 = axs[2].twiny()  
    ax1.plot(log_df.AT10, log_df.index, '-', color='#6e787c', linewidth=0.5)
    ax1.set_xlim(0.1,100000)  
    ax1.set_xlabel('AT10 (ohm.m)', color='#6e787c')
    ax1.set_xscale('log')
    ax1.minorticks_on()
    ax1.spines['top'].set_position(('axes', 1.075))
    
    ax2 = axs[2].twiny()  
    ax2.plot(log_df.AT90, log_df.index, '-', color='#ea0606', linewidth=0.5)
    ax2.set_xlim(0.1,100000)  
    ax2.set_xlabel('AT90 (ohm.m)', color='#ea0606')
    ax2.set_xscale('log')
    ax2.minorticks_on()
    ax2.spines['top'].set_position(('axes', 1.0))
    ax2.grid(True)
    axs[2].get_xaxis().set_visible(False)


  # Fourth track XRD to display
    ax1 = axs[3].twiny()  
    ax1.plot(cuttings_df.Qz, cuttings_df.index, 'o', color='#eac406')
    ax1.set_xlim(0,50) 
    ax1.set_xlabel('Quartz %', color='#eac406')
    ax1.minorticks_on()
    ax1.spines['top'].set_position(('axes', 1.075))
    
    ax2 = axs[3].twiny()  
    ax2.plot(cuttings_df.Illi, cuttings_df.index, 'o', color='#94898c')
    ax2.set_xlim(0,50) 
    ax2.set_xlabel('Illite %', color='#94898c')
    ax2.minorticks_on()
    ax2.spines['top'].set_position(('axes', 1.0))
    ax2.grid(True)
    axs[3].get_xaxis().set_visible(False)


    # Fifth track Temp/TC to display
    ax1 = axs[4].twiny()  
    ax1.plot(cuttings_df.TC, cuttings_df.index, 'o', color='#6e787c')
    ax1.set_xlim(0,5) 
    ax1.set_xlabel('Matrix TC Measured W/mC', color='#6e787c')
    ax1.minorticks_on()
    ax1.spines['top'].set_position(('axes', 1.075))

    ax2 = axs[4].twiny()  
    ax2.plot(log_df.CTEM_C, log_df.index, '-', color='#ed8712')
    ax2.set_xlim(20,200) 
    ax2.set_xlabel('Temp degC', color='#ed8712')
    ax2.minorticks_on()
    ax2.spines['top'].set_position(('axes', 1.0))
    ax2.grid(True)
    axs[4].get_xaxis().set_visible(False)
         
    fig.suptitle('Well Data for UTAH FORGE 58-32',weight='bold', fontsize=20, y=0.85);
    plt.show()


In [ ]:
#Applying the data frames, that we have created above, to the function make_layout_tc (inputs)
make_layout_tc (log_df, cuttings_df)

## We are Done